In [3]:
from sagas.nlu.stanza_helper import get_nlp
from spacy_stanza import StanzaLanguage

snlp = get_nlp('ru')
nlp = StanzaLanguage(snlp)

# doc = nlp('У Вас есть сигареты?')
doc = nlp('У Вас есть Основной номер факса?')
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)
print('*'*25)
print(doc.ents)

У у ADP case 
Вас вы PRON obl MISC
есть быть VERB root MISC
Основной основной ADJ amod MISC
номер номер NOUN nsubj MISC
факса факс NOUN nmod MISC
? ? PUNCT punct 
*************************
(Вас есть Основной номер факса,)


In [4]:
from spacy import displacy
displacy.render([doc], style="dep")

In [4]:
import spacy
from spacy.pipeline import EntityRuler
from spacy_stanza import StanzaLanguage
from sagas.nlu.stanza_helper import get_nlp

snlp = get_nlp('ru')
nlp = StanzaLanguage(snlp)
ruler = EntityRuler(nlp, overwrite_ents=True, validate=True)
patterns = [{"label": "typ", "pattern": "Основной номер факса", 'id': "ContactMechPurposeType.description.FAX_NUMBER"}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

2020-04-22 03:59:06,312 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/pymorphy2_dicts/data
2020-04-22 03:59:06,482 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/runpy.py:193: UserWarning: [W012] A Doc object you're adding to the PhraseMatcher for pattern 'typ||ContactMechPurposeType.description.FAX_NUMBER' is parsed and/or tagged, but to match on 'ORTH', you don't actually need this information. This means that creating the patterns is potentially much slower, because all pipeline components are applied. To only create tokenized Doc objects, try using `nlp.make_doc(text)` or process all texts as a stream using `list(nlp.tokenizer.pipe(all_texts))`.
  "__main__", mod_spec)


In [12]:
all_texts=["Основной номер факса"]
list(nlp.tokenizer.pipe(all_texts))

[Основной номер факса]

In [8]:
nlp.pipe_names

['entity_ruler']

In [7]:
doc = nlp('У Вас есть Основной номер факса ?')
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc.ents])
print([(tok.i, [tok.ent_type_, tok.ent_iob_]) for tok in doc])

[('Основной номер факса', 'typ', 'ContactMechPurposeType.description.FAX_NUMBER')]
[(0, ['', '']), (1, ['', 'O']), (2, ['', 'O']), (3, ['typ', 'B']), (4, ['typ', 'I']), (5, ['typ', 'I']), (6, ['', ''])]


In [8]:
from spacy import displacy
displacy.render([doc], style="dep")

In [18]:
'Основной номер факса'.lower()

'основной номер факса'

In [19]:
from spacy.matcher import PhraseMatcher
nlp = StanzaLanguage(snlp)
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
# matcher = PhraseMatcher(nlp.vocab)
terms = ["Основной номер факса"]
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", None, *patterns)

# doc = nlp('У Вас есть Основной номер факса?')
doc = nlp('У Вас есть основной номер факса?')
matches = matcher(doc)
spans=[]
for match_id, start, end in matches:
    span = doc[start:end]
    match_id_string = nlp.vocab.strings[match_id]
    print(match_id_string, '->', span.text)
    spans.append(span)

with doc.retokenize() as retokenizer:
    for span in spans:
        retokenizer.merge(span)

TerminologyList -> основной номер факса


In [11]:
from spacy import displacy
displacy.render([doc], style="dep")

In [14]:
for word in doc:
    if word.dep_ in ("obl"):
        print("".join(w.text_with_ws for w in word.subtree))

У Вас 


In [23]:
print([(tok.i, tok.text, token.head.i) for tok in doc])
print([(tok.i, [tok.ent_type_, tok.ent_iob_]) for tok in doc])

[(0, 'У', 2), (1, 'Вас', 2), (2, 'есть', 2), (3, 'основной номер факса', 2), (4, '?', 2)]
[(0, ['', '']), (1, ['', '']), (2, ['', '']), (3, ['', '']), (4, ['', ''])]
